# Lazy Wave Riser Configuration Optimization

In [20]:
# import the required libreries
import OrcFxAPI as api
import scipy.optimize as opt
import numpy as np
import matplotlib.pyplot as plt

In [21]:
# initilize the model
mdl = api.Model()

In [23]:
# define general and environment
mdl.general.NorthDirectionDefined, mdl.general.NorthDirection = 'Yes', 90
mdl.environment.WaterDepth = 400

In [24]:
# create object tuypes
riserType = mdl.CreateObject(api.otLineType, name='Riser Type')
riserType.OD, riserType.ID, riserType.MassPerUnitLength = 0.356, 0.254, 0.184
riserType.AllowableTension, riserType.xMinRadius = 5000, 3.675
riserType.EIx, riserType.EA, riserType.PoissonRatio, riserType.GJ = 124.869, 711.2E3, 0.5, 10

floatType = mdl.CreateObject(api.otLineType, name='Float Type')
floatType.WizardCalculation = 'Line with Floats'
floatType.FloatBaseLineType = riserType.Name
floatType.FloatDiameter = 1.2
floatType.FloatLength = 1
floatType.FloatPitch = 3
floatType.InvokeWizard()

In [41]:
floatType.PenWidth, floatType.PenColour = 3, 16777088

In [25]:
# craete the objects
vessel = mdl.CreateObject(api.otVessel, name='FPSO')
vessel.InitialX, vessel.InitialY, vessel.InitialZ, vessel.InitialHeading = 0, 0, 0, 180

riser = mdl.CreateObject(api.otLine, name='Lazy Wave Riser')
riser.LineType = riserType.Name, riserType.Name, floatType.Name, riserType.Name, riserType.Name
riser.Length = 200, 100, 200, 150, 100
riser.TargetSegmentLength = 4, 1, 1, 1, 4

riser.EndAConnection = vessel.Name
riser.EndAX, riser.EndAY, riser.EndAZ, riser.EndAAzimuth, riser.EndADeclination = -2, -3.4, -7.5, 60, 170

riser.EndBConnection = 'Anchored'
riser.EndBX, riser.EndBY, riser.EndBHeightAboveSeabed, riser.EndBDeclination = 150, 320, 0, 90

riser.ContentsDensity = 1

riser.EndAxBendingStiffness, riser.EndBxBendingStiffness = api.OrcinaInfinity(), api.OrcinaInfinity()
riser.LayAzimuth = 245

In [42]:
riser.ContactPenWidth, riser.ContactPenColour = 5, 16777215

In [43]:
mdl.SaveData('opt2.yml')

### End of modelling -----------------------------------------------

In [44]:
# target
sagbendZ, sagbendR = -300, 70                                  # B
hogbendZ, hogbendR = -270, 115                                 # C
TDZ, TDR, TDX, TDY = -mdl.environment.WaterDepth, 200, 85, 180 # D

In [47]:
x = [100, 200, 3, 150]

In [48]:
# objective function
def objF(x):
    riser.Length[1], riser.Length[2], floatType.FloatPitch, riser.Length[3] = x
    
    floatType.InvokeWizard()
    mdl.CalculateStatics()
    
    # sagbend B
    arc1 = api.arSpecifiedSections(1, 2)
    x1 = riser.RangeGraph('X', arclengthRange=arc1).Mean
    y1 = riser.RangeGraph('Y', arclengthRange=arc1).Mean
    z1 = riser.RangeGraph('Z', arclengthRange=arc1).Mean
    
    sagbendZi = min(z1)
    i1 = np.argmin(z1)
    sagbendRi = np.sqrt(x1[i1]**2 + y1[i1]**2)
    
    # hogbend C
    arc2 = api.arSpecifiedSections(3, 3)
    x2 = riser.RangeGraph('X', arclengthRange=arc2).Mean
    y2 = riser.RangeGraph('Y', arclengthRange=arc2).Mean
    z2 = riser.RangeGraph('Z', arclengthRange=arc2).Mean

    hogbendZi = max(z2)
    i2 = np.argmax(z2)
    hogbendRi = np.sqrt(x2[i2]**2 + y2[i2]**2)

    # touchdown D
    arc3 = api.arSpecifiedSections(4, 5)
    x3 = riser.RangeGraph('X', arclengthRange=arc3).Mean
    y3 = riser.RangeGraph('Y', arclengthRange=arc3).Mean
    z3 = riser.RangeGraph('Z', arclengthRange=arc3).Mean

    iRested = np.where(z3 - riserType.OD/2 <= -mdl.environment.WaterDepth) # indices of rested nodes on the seabed
    i3 = iRested[0][0] # index of TDP
    TDZi = z3[i3] - riserType.OD/2
    TDRi = np.sqrt(x3[i3]**2 + y3[i3]**2)
    TDXi = x3[i3]
    TDYi = y3[i3]
    
    obj2 = ((sagbendZ - sagbendZi)**2 + (sagbendR - sagbendRi)**2 + (hogbendZ - hogbendZi)**2 +
            (hogbendR - hogbendRi)**2 + (TDZ - TDZi)**2 + (TDR - TDRi)**2 + (TDX - TDXi)**2 + (TDY - TDYi)**2)
    
    print('{:7.1f} {:7.1f} {:9.1f} {:8.1f} {:7.1f} {:6.1f} {:6.1f} {:6.1f}{:7.1f} {:8.1f}{:9.1f}{:10.1f}{:14.5f}'.format(
        sagbendZi,sagbendRi,hogbendZi,hogbendRi,TDZi,TDRi,TDXi,TDYi,x[0],x[1],x[2],x[3], obj2))
        
    return obj2    

In [49]:
# test the objective function
x = [100, 200, 3, 150]
objF(x)

 -265.4    46.9    -204.1    101.3  -400.0  184.4   78.6  166.8  100.0    200.0      3.0     150.0    6719.27914


6719.2791446619667

In [50]:
# initial guess
x0 = np.ndarray(4)
x0[:] = [riser.Length[1], riser.Length[2], floatType.FloatPitch, riser.Length[3]]

# vars bounds
bnds = ((10,150), (10,300), (0.5,4), (10,200))


print('{:>5} {:>5} {:>5} {:>5} {:>5} {:>6} {:>6} {:>6} {:>8} {:>8} {:>10}{:>8}{:>8}'.format('sagbendZ', 'sagbendR',
    'hogbendZ', 'hogbendR', 'TDZ', 'TDR', 'TDX', 'TDY', 'Length1', 'Length2', 'FloatPitch', 'Length3','objF'))

# call scipy to minimize the objective function
opt.minimize(objF, x0, method='SLSQP', bounds=bnds) # only for L-BFGS-B, TNC and SLSQP

sagbendZ sagbendR hogbendZ hogbendR   TDZ    TDR    TDX    TDY  Length1  Length2 FloatPitch Length3    objF
 -265.4    46.9    -204.1    101.3  -400.0  184.4   78.6  166.8  100.0    200.0      3.0     150.0    6719.27914
 -265.4    46.9    -204.1    101.3  -400.0  184.4   78.6  166.8  100.0    200.0      3.0     150.0    6719.27914
 -265.4    46.9    -204.1    101.3  -400.0  184.4   78.6  166.8  100.0    200.0      3.0     150.0    6719.27914
 -265.4    46.9    -204.1    101.3  -400.0  184.4   78.6  166.8  100.0    200.0      3.0     150.0    6719.27915
 -265.4    46.9    -204.1    101.3  -400.0  184.4   78.6  166.8  100.0    200.0      3.0     150.0    6719.27904
 -265.4    46.9    -204.1    101.3  -400.0  184.4   78.6  166.8  100.0    200.0      3.0     150.0    6719.27915
 -329.3   123.0    -329.3    123.0  -400.0  235.5  100.2  213.2  150.0     80.4      4.0      72.5    9846.54680
 -306.6    50.5    -287.6     91.1  -400.0  178.6   76.1  161.6  122.8    145.6      3.5     114.7   

     jac: array([ 0.00070263, -0.00103459,  0.06250439, -0.00017922,  0.        ])
    njev: 23
       x: array([ 138.08247606,  115.56305395,    2.43570566,  133.00296789])
 success: True
     fun: 0.78190191004131526
 message: 'Optimization terminated successfully.'
    nfev: 155
     nit: 23
  status: 0

In [51]:
x = np.ndarray(4)
x[:] = riser.Length[1], riser.Length[2], floatType.FloatPitch, riser.Length[3]
x

array([ 138.08247606,  115.56305395,    2.43570566,  133.00296791])

In [52]:
x[:] = [138, 116, 2.436, 133]
print('{:>5} {:>5} {:>5} {:>5} {:>5} {:>6} {:>6} {:>6} {:>8} {:>8} {:>10}{:>8}{:>8}'.format('sagbendZ', 'sagbendR',
    'hogbendZ', 'hogbendR', 'TDZ', 'TDR', 'TDX', 'TDY', 'Length1', 'Length2', 'FloatPitch', 'Length3','objF'))
objF(x)

print('{:7.1f} {:7.1f} {:9.1f} {:8.1f} {:7.1f} {:6.1f} {:6.1f} {:6.1f} {:>15s}'.format(
    sagbendZ,sagbendR,hogbendZ,hogbendR,TDZ,TDR,TDX,TDY, '<----- Target'))

sagbendZ sagbendR hogbendZ hogbendR   TDZ    TDR    TDX    TDY  Length1  Length2 FloatPitch Length3    objF
 -299.8    68.8    -269.5    115.5  -400.0  199.4   84.9  180.4  138.0    116.0      2.4     133.0       2.46864
 -300.0    70.0    -270.0    115.0  -400.0  200.0   85.0  180.0   <----- Target


In [53]:
mdl.SaveData('opt3.yml')